#**Taller Cifrado Grille**

Gabriel Santiago Delgado Lozano

Nota: Para evitar conflictos, una matriz con dimensión impar siempre tendrá una X en el centro al encriptar. Al desencriptar, se va a ignorar completamente cualquier valor que esté en dicha coordenada. Sin embargo, la 'X' debe estar presente en el mensaje encriptado para tener una cadena válida. Adicionalmente, un texto cifrado que sea válido debe tener una longitud que sea múltiplo del área de la matriz, o sea, $n^2$. La longitud del texto no tiene restricciones al encriptar, ya que en caso de ser necesario, se partirá el mensaje en bloques y/o se hará padding.

In [3]:
import numpy as np
import re

def holeParsing(holeString, size, direction):
    try:
        holes = [tuple(map(int, coord.split(','))) for coord in holeString.split(';') if coord.strip()]
        rotations = holeValidation(size, holes, direction)
        if rotations:
            return rotations
        else:
            return False
    except ValueError:
        print("Formato inválido. Ingrese las coordenadas de la forma 'x1,y1;x2,y2;...'")
        return False

def holeValidation(size, holes, direction):
    rotations = computeRotations(size, holes, direction)
    center = None
    if size % 2 != 0:
        center = (size // 2, size // 2)
    visited = []
    for rotation in rotations:
        for x, y in rotation:
            if (x, y) == center:
                continue
            elif x < 0 or x >= size or y < 0 or y >= size:
                print(f"Coordenada ({x},{y}) inválida. Ingrese coordenadas válidas.")
                return False
            elif (x, y) in visited:
                print("Coordenadas repetidas. Ingrese coordenadas válidas.")
                return False
            else:
              visited.append((x, y))
    if size % 2 != 0:
      if len(visited) != size**2 - 1:
        print("No se cubren todos los espacios. Por favor ingrese nuevas coordenadas.")
        return False
    else:
      if len(visited) != size**2:
        print("No se cubren todos los espacios. Por favor ingrese nuevas coordenadas.")
        return False
    return rotations

def computeRotations(matrixSize, positions, direction):
    rotations = [positions]
    for _ in range(3):
        newPositions = []
        for x, y in rotations[-1]:
            if direction == 1:
                newX = y
                newY = matrixSize - 1 - x
            else:
                newX = matrixSize - 1 - y
                newY = x
            newPositions.append((newX, newY))
        newPositions.sort()
        rotations.append(newPositions)
    return rotations


def grilleEncryption(matrixSize, direction, plaintext, rotations):
    cleanedPlaintext = re.sub(r'[^a-zA-Z]', '', plaintext.upper())
    matrixArea = matrixSize**2
    if matrixSize % 2 == 0:
        plaintextChunks = [cleanedPlaintext[i:i + matrixArea] for i in range(0, len(cleanedPlaintext), matrixArea)]
    else:
        plaintextChunks = [cleanedPlaintext[i:i + matrixArea - 1] for i in range(0, len(cleanedPlaintext), matrixArea - 1)]
    #print(rotations)
    ciphertext = ""
    center = None
    if matrixSize % 2 == 1:
        center = (matrixSize // 2, matrixSize // 2)
    for messageChunk in plaintextChunks:
        if len(messageChunk) < matrixArea:
            messageChunk = messageChunk.ljust(matrixArea, 'X')
        grille = np.full((matrixSize, matrixSize), ' ', dtype=str)
        messageChunkIndex = 0
        for rotation in rotations:
            for x, y in rotation:
              if (x,y) == center:
                grille[x, y] = 'X'
              else:
                grille[x, y] = messageChunk[messageChunkIndex]
                messageChunkIndex += 1
              #print(grille)
        if matrixSize % 2 == 1:
            grille[center] = 'X'
        for row in grille:
            ciphertext += ''.join(row)
    return ciphertext

def grilleDecryption(matrixSize, direction, ciphertext, rotations):
    cleanedCiphertext = re.sub(r'[^a-zA-Z]', '', ciphertext.upper())
    matrixArea = matrixSize**2
    ciphertextChunks = [cleanedCiphertext[i:i + matrixArea] for i in range(0, len(cleanedCiphertext), matrixArea)]
    print(ciphertextChunks)
    center = None
    if matrixSize % 2 == 1:
        center = (matrixSize // 2, matrixSize // 2)
    plaintext = ""
    for cipherChunk in ciphertextChunks:
      charList = list(cipherChunk)
      grille = np.array(charList).reshape(matrixSize, matrixSize)
      #print(grille)
      for rotation in rotations:
        for x, y in rotation:
          if (x,y) == center:
            continue
          else:
            plaintext += str(grille[x, y])
    return plaintext

# holes = "1,0;1,1;2,0"
# positions = holeParsing(holes, 3, 0)
# if (positions != False):
#   ciphertext = grilleEncryption(3,0,'HELLOWORLD',positions)
#   print(ciphertext)
#   plaintext = grilleDecryption(3,0,ciphertext,positions)
#   print(plaintext)
# else:
#   print("Yuca")
while True:
  flag = input("Ingrese 1 si desea encriptar, 0 si desea desencriptar, 2 si desea salir: ")
  match flag:
    case "1":
      size = int(input("Ingrese el tamaño de la matriz: "))
      message = input("Ingrese el texto a encriptar: ")
      print("Ingrese la direccion para rotar la grilla, 1 a favor de las manecillas y 0 en contra:")
      direccion = int(input())
      if (direccion != 0 and direccion != 1):
        print("Ingrese una direccion valida")
        continue
      print("Ingrese las coordenadas (x,y) de los huecos separados por punto y coma de la siguiente forma: 'x1,y1;x2,y2;...'")
      holes = input()
      positions = holeParsing(holes, size, direccion)
      if (positions == False):
        continue
      #print(positions)
      ciphertext = grilleEncryption(size, direccion, message, positions)
      print("Mensaje encriptado: ", ciphertext)
      print("-------------------------------------------------------------------")
    case "0":
      size = int(input("Ingrese el tamaño de la matriz: "))
      message = input("Ingrese el texto a desencriptar: ")
      if len(message) % size**2 != 0:
        print("Longitud inválida para el texto cifrado. Por favor ingrese un texto cifrado válido.")
        continue
      print("Ingrese la direccion para rotar la grilla, 1 a favor de las manecillas y 0 en contra:")
      direccion = int(input())
      if (direccion != 0 and direccion != 1):
        print("Ingrese una direccion valida")
        continue
      print("Ingrese las coordenadas (x,y) de los huecos separados por punto y coma de la siguiente forma: 'x1,y1;x2,y2;...'")
      holes = input()
      positions = holeParsing(holes, size, direccion)
      if (positions == False):
        continue
      #print(positions)
      plaintext = grilleDecryption(size, direccion, message, positions)
      print("Mensaje desencriptado: ", plaintext)
      print("-------------------------------------------------------------------")
    case "2":
      print("Chao :)")
      break
    case _ :
      print("Ingrese una opcion valida")
      print("-------------------------------------------------------------------")

Ingrese 1 si desea encriptar, 0 si desea desencriptar, 2 si desea salir: 1
Ingrese el tamaño de la matriz: 5
Ingrese el texto a encriptar: You spin me right round
Ingrese la direccion para rotar la grilla, 1 a favor de las manecillas y 0 en contra:
1
Ingrese las coordenadas (x,y) de los huecos separados por punto y coma de la siguiente forma: 'x1,y1;x2,y2;...'
1,0;1,4;2,0;2,3;3,3;4,0
Mensaje encriptado:  NDMEHYTXXOURXSOURIPNIXXGX
-------------------------------------------------------------------
Ingrese 1 si desea encriptar, 0 si desea desencriptar, 2 si desea salir: 0
Ingrese el tamaño de la matriz: 5
Ingrese el texto a desencriptar: NDMEHYTXXOURXSOURIPNIXXGX
Ingrese la direccion para rotar la grilla, 1 a favor de las manecillas y 0 en contra:
1
Ingrese las coordenadas (x,y) de los huecos separados por punto y coma de la siguiente forma: 'x1,y1;x2,y2;...'
1,0;1,4;2,0;2,3;3,3;4,0
['NDMEHYTXXOURXSOURIPNIXXGX']
Mensaje desencriptado:  YOUSPINMERIGHTROUNDXXXXX
---------------------------